<a href="https://colab.research.google.com/github/Chubbyman2/SSD_MobileNet_Hand_Tracker/blob/main/training_ssd_mobilenet_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Original Code From:

https://medium.com/@matus.tanon/custom-object-detection-using-tensorflow-in-google-colab-e4d6e1a17f18

This will provide you with the frozen_inference_graph.pb needed for the prediction program.

In [41]:
%tensorflow_version 1.x  # Only runs on ver 1.x of Tensorflow
!pip show tensorflow

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `1.x  # Only runs on ver 1.x of Tensorflow`. This will be interpreted as: `1.x`.


TensorFlow is already loaded. Please restart the runtime to change versions.
Name: tensorflow
Version: 1.15.2
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /tensorflow-1.15.2/python3.6
Requires: tensorflow-estimator, numpy, keras-applications, keras-preprocessing, protobuf, tensorboard, termcolor, wheel, gast, absl-py, astor, six, grpcio, wrapt, google-pasta, opt-einsum
Required-by: stable-baselines, magenta, fancyimpute


In [42]:
!pip install tf_slim

In [43]:
%cd /root/
!git clone https://github.com/tensorflow/models.git # Import required models from Github

/root
fatal: destination path 'models' already exists and is not an empty directory.


In [44]:
# Install Tensorboard
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -o ngrok-stable-linux-amd64.zip

--2021-01-02 00:55:38--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 54.209.129.196, 34.198.20.103, 52.2.56.23, ...
Connecting to bin.equinox.io (bin.equinox.io)|54.209.129.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13773305 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip.1’

ngrok-stable-linux- 100%[===================>]  13.13M  14.4MB/s    in 0.9s    

2021-01-02 00:55:39 (14.4 MB/s) - ‘ngrok-stable-linux-amd64.zip.1’ saved [13773305/13773305]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [45]:
# The logs that are created while training 
LOG_DIR = "/root/models/trained"
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)
get_ipython().system_raw('./ngrok http 6006 &')

# The link to tensorboard works after the training starts
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

Traceback (most recent call last):
  File "<string>", line 1, in <module>
IndexError: list index out of range


In [46]:
# Quick test of the Object Detection API
%cd /root/models/research/
!protoc object_detection/protos/*.proto --python_out=.
import os
os.environ['PYTHONPATH'] += ':/root/models/research/:/root/models/research/slim/:/root/models/research/object_detection/utils/:/root/models/research/object_detection'
!python object_detection/builders/model_builder_test.py

/root/models/research


In [47]:
# Download the pre-trained ssd_mobilenet_v1_coco model from Tensorflow model zoo
%cd ~/models

import os
import shutil
import glob
import urllib
import tarfile
from requests import get

MODEL = 'ssd_mobilenet_v1_coco_2017_11_17'
MODEL_FILE = MODEL + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'
DEST_DIR = 'pretrained_model'

if not (os.path.exists(MODEL_FILE)):
  with open(MODEL_FILE, "wb") as file:
    response = get(DOWNLOAD_BASE + MODEL_FILE)
    file.write(response.content)

tar = tarfile.open(MODEL_FILE)
tar.extractall()
tar.close()

os.remove(MODEL_FILE)
if (os.path.exists(DEST_DIR)):
  shutil.rmtree(DEST_DIR)
os.rename(MODEL, DEST_DIR)

/root/models


In [52]:
# Move Config file to directory "models"
try:
  shutil.move("/root/models/research/object_detection/samples/configs/ssd_mobilenet_v1_coco.config", "/root/models")
except:
  pass

In [57]:
%cd /root/models/research/
!protoc object_detection/protos/*.proto --python_out=.
import os
os.environ['PYTHONPATH'] += ':/root/models/research/:/root/models/research/slim/:/root/models/research/object_detection/utils/:/root/models/research/object_detection'

# Edit Pipeline 
import tensorflow as tf
from google.protobuf import text_format
from object_detection.protos import pipeline_pb2

pipeline = pipeline_pb2.TrainEvalPipelineConfig()                                                                                                                                                                                                          
config_path = '/root/models/ssd_mobilenet_v1_coco.config'
with tf.gfile.GFile(config_path, "r") as f:                                                                                                                                                                                                                     
    proto_str = f.read()                                                                                                                                                                                                                                          
    text_format.Merge(proto_str, pipeline)

pipeline.train_config.fine_tune_checkpoint = '/root/models/pretrained_model/model.ckpt'
pipeline.train_config.num_steps = 500
pipeline.model.ssd.num_classes = 1 # Just hands, nothing else
pipeline.eval_config.num_examples = 5

/root/models/research


In [ ]:
import zipfile
from google.colab import drive
drive.mount('/content/drive/')

!unzip "/content/drive/My Drive/egohands_images.zip"

In [75]:
# Change the first four lines to where the images and label maps are
pipeline.train_input_reader.tf_record_input_reader.input_path[0] = 'egohands_images/train/train.record'
pipeline.train_input_reader.label_map_path = '/content/drive/My Drive/hand_label_map.pbtxt'
pipeline.eval_input_reader[0].tf_record_input_reader.input_path[0] = 'egohands_images/test/test.record'
pipeline.eval_input_reader[0].label_map_path = '/content/drive/My Drive/hand_label_map.pbtxt'

config_text = text_format.MessageToString(pipeline)                                                                                                                                                                                                        
with tf.gfile.Open(config_path, "wb") as f:                                                                                                                                                                                                                       
    f.write(config_text)

In [76]:
# Compile protobuf and change Python path
%cd /root/models/research/
!protoc object_detection/protos/*.proto --python_out=.
import os
os.environ['PYTHONPATH'] += ':/root/models/research/:/root/models/research/slim/:/root/models/research/object_detection/utils/:/root/models/research/object_detection'

# Begin training
!python /root/models/research/object_detection/legacy/train.py \
    --logtostderr \
    --train_dir=/root/models/trained \
    --pipeline_config_path=/root/models/ssd_mobilenet_v1_coco.config

/root/models/research
Instructions for updating:
Use object_detection/model_main.py.
W0102 01:14:50.641551 139624809736064 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/absl/app.py:251: main (from __main__) is deprecated and will be removed in a future version.
Instructions for updating:
Use object_detection/model_main.py.
Instructions for updating:
Please switch to tf.train.create_global_step
W0102 01:14:50.657610 139624809736064 deprecation.py:323] From /root/models/research/object_detection/legacy/trainer.py:265: create_global_step (from tf_slim.ops.variables) is deprecated and will be removed in a future version.
Instructions for updating:
Please switch to tf.train.create_global_step
INFO:tensorflow:Reading unweighted datasets: ['egohands_images/train/train.record']
I0102 01:14:50.681065 139624809736064 dataset_builder.py:148] Reading unweighted datasets: ['egohands_images/train/train.record']
INFO:tensorflow:Reading record datasets for input file: ['egohands_imag

In [78]:
# Export trained model 
%cd /root/models/research
!protoc object_detection/protos/*.proto --python_out=.
import os
os.environ['PYTHONPATH'] += ':/root/models/research/:/root/models/research/slim/:/root/models/research/object_detection/utils/:/root/models/research/object_detection'
!python /root/models/research/object_detection/export_inference_graph.py \
    --input_type=image_tensor \
    --pipeline_config_path=/root/models/ssd_mobilenet_v1_coco.config \
    --output_directory=/root/models/fine_tuned_model \
    --trained_checkpoint_prefix=/root/models/trained/model.ckpt-500

/root/models/research
Instructions for updating:
Please use `layer.__call__` method instead.
W0102 01:38:26.729327 140533598230400 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tf_slim/layers/layers.py:1089: Layer.apply (from tensorflow.python.keras.engine.base_layer) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `layer.__call__` method instead.
INFO:tensorflow:depth of additional conv before box predictor: 0
I0102 01:38:28.536709 140533598230400 convolutional_box_predictor.py:156] depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
I0102 01:38:28.583788 140533598230400 convolutional_box_predictor.py:156] depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
I0102 01:38:28.628886 140533598230400 convolutional_box_predictor.py:156] depth of additional conv before box predictor: 0
INFO:tensorflow:depth o